# Image Detection Demo with PytorchWildlife

This tutorial guides you on how to use PyTorchWildlife to separate positive and negative animal detections. We will go through the process of setting up the environment, defining the detection model, as well as performing inference and saving the results in different ways.

## Prerequisites
Install PytorchWildlife running the following commands:
```bash
conda create -n pytorch_wildlife python=3.8 -y
conda activate pytorch_wildlife
pip install PytorchWildlife
```
Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. This notebook can also run on CPU.

## Importing libraries
First, we'll start by importing the necessary libraries and modules.

In [ ]:
import os
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

## Model Initialization
We will initialize the MegaDetectorV5 model for image detection. This model is designed for detecting animals in images.

In [ ]:
# Setting the device to use for computations ('cuda' indicates GPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
if DEVICE == "cuda":
    torch.cuda.set_device(0)
    
# Initializing the MegaDetectorV6 model for image detection
detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="yolov9c")

# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
#detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")


## Variable definition
In order to process the batch detection, we will define an input directory where the images are stored, a confidence threshold and an output directory to copy the positive and negative images into distinctive folders. If you want to follow this tutorial with your own data, modify the following variables.

In [ ]:
tgt_folder_path = os.path.join(".","demo_data","imgs")
output_path = "folder_separation"
threshold = 0.2

## Batch Image Detection
Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.

In [ ]:
results = detection_model.batch_image_detection(tgt_folder_path, batch_size=16)

## Separate positive and negative detections
PytorchWildlife allows to copy the files from your original folder to a new directory containing the "Animal" and "No-animal" subdirectories. A detection is considered positive if the prediction confidence is higher than the threshold

In [ ]:
os.makedirs(output_path, exist_ok=True)
json_file = os.path.join(output_path, "detection_results.json")
pw_utils.save_detection_json(results, json_file,
                             categories=detection_model.CLASS_NAMES,
                             exclude_category_ids=[], # Category IDs can be found in the definition of each model.
                             exclude_file_path=tgt_folder_path)

# Separate the positive and negative detections through file copying:
pw_utils.detection_folder_separation(json_file, tgt_folder_path, output_path, threshold)

### Copyright (c) Microsoft Corporation. All rights reserved.
### Licensed under the MIT License.